In [ ]:
import streamlit as st
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import streamlit.components.v1 as components
from var import areaCd, unitySignguCd, preferCnttgClscd, spctrCd, spctrSecd as spctrSecd_dict
import utils

st.set_page_config(layout="wide")
st.title("📦 지역 기반 보유 수량 시각화 대시보드")

# ------------------------
# 🧭 탭 구성
# ------------------------
tab1, tab2 = st.tabs(["📊 지역구 지도 및 물품 통계", "🏆 센터별 Top 5 보유량"])

# ------------------------
# 📊 Tab 1: 지역구 지도 + 물품 barplot
# ------------------------
with tab1:
    col1, col2 = st.columns([1,2])
    with col1:
        selected_sido = st.selectbox("시도 선택 (Tab1)", list(areaCd.values()), key="tab1_sido")
        selected_sido_code = [k for k, v in areaCd.items() if v == selected_sido][0]

    with col2:
        sigungu_options = {k: v for k, v in unitySignguCd.items() if k.startswith(selected_sido_code)}
        selected_sigungu_name = st.selectbox("지역구 선택", list(sigungu_options.values()), key="tab1_sigungu")
        selected_sigungu_code = [k for k, v in sigungu_options.items() if v == selected_sigungu_name][0]

    # 센터 정보 로드
    @st.cache_data
    def load_centers():
        return pd.read_csv("./data/food_centers_with_region_and_coords.csv", encoding="utf-8", dtype=object).dropna(subset=["위도", "경도"])

    centers_df = load_centers()
    filtered_df = centers_df[(centers_df["areaCd"] == selected_sido_code) & (centers_df["unitySignguCd"] == selected_sigungu_code)]

    col1, col2 = st.columns([1, 2])
    with col1:
        st.markdown("### 🗺️ 푸드뱅크/마켓 위치 지도")
        centers_df_map = filtered_df.copy()
        centers_df_map[["위도", "경도"]] = centers_df_map[["위도", "경도"]].astype(float)
        st.map(centers_df_map.rename(columns={"위도": "lat", "경도": "lon"}))

        # # Leaflet용 JSON 생성
        # food_centers = [
        #     {
        #         "label": row["spctrCd"],
        #         "name": spctrCd.get(row["spctrCd"], row["spctrCd"]),
        #         "address": row["spctrAdres"],
        #         "phone": row["spctrTelno"],
        #         "lat": row["위도"],
        #         "lon": row["경도"],
        #         "sido": areaCd.get(row["areaCd"], row["areaCd"]),
        #         "sigungu": unitySignguCd.get(row["unitySignguCd"], row["unitySignguCd"]),
        #         "type": spctrSecd_dict.get(row["spctrSecd"], row["spctrSecd"])
        #     }
        #     for _, row in filtered_df.iterrows()
        # ]

        # food_centers_json = json.dumps(food_centers, ensure_ascii=False)

        # html_string = f"""
        # <!DOCTYPE html>
        # <html><head><meta charset='utf-8' />
        # <link rel='stylesheet' href='https://unpkg.com/leaflet@1.9.3/dist/leaflet.css' />
        # <script src='https://unpkg.com/leaflet@1.9.3/dist/leaflet.js'></script>
        # <style>#map {{ height: 650px; border-radius: 12px; }}</style></head>
        # <body><div id='map'></div><script>
        # const map = L.map('map').setView([36.5, 127.8], 7);
        # L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png').addTo(map);
        # const centers = {food_centers_json};
        # centers.forEach(c => {{
        #     const popup = `<b>${{c.name}}</b><br/>주소: ${{c.address}}<br/>전화: ${{c.phone}}<br/><i>${{c.sido}} ${{c.sigungu}}</i><br/><span style='color:gray;'>[${{c.type}}]</span>`;
        #     let color = c.type === "푸드마켓" ? "#28a745" : c.type === "물류센터" ? "#fd7e14" : "#007BFF";
        #     L.circleMarker([c.lat, c.lon], {{ radius: 6, color, fillColor: color, fillOpacity: 0.9 }}).bindPopup(popup).addTo(map);
        # }});
        # if (centers.length > 0) {{
        #     const group = new L.featureGroup(centers.map(c => L.marker([c.lat, c.lon])));
        #     map.fitBounds(group.getBounds());
        # }}
        # </script></body></html>
        # """
        # components.html(html_string, height=650)

    # 📊 보유수량 시각화
    with col2:
        col2_1, col2_2 = st.columns(2)
        with col2_1:
            @st.cache_data
            def load_prefer_by_region(area_cd, unity_signgu_cd):
                records = []
                for prefer_code, prefer_name in preferCnttgClscd.items():
                    df = utils.getPreferInfo(prefer_cnttg_clscd=prefer_code, area_cd=area_cd, unity_signgu_cd=unity_signgu_cd)
                    df["물품명"] = prefer_name
                    df["보유수량"] = df["holdQy"]
                    records.append(df[["물품명", "보유수량"]])
                return pd.concat(records)

            df = load_prefer_by_region(area_cd=selected_sido_code, unity_signgu_cd=selected_sigungu_code)
            if not df.empty:
                df_grouped = df.groupby("물품명")["보유수량"].sum().reset_index().sort_values("보유수량", ascending=False)
                st.markdown(f"### 📈 {selected_sigungu_name} 지역 보유 상위 품목")
                st.dataframe(df_grouped,
                            column_order=("물품명", "보유수량"),
                            hide_index=True,
                            use_container_width=True,
                            column_config={
                                "물품명": st.column_config.TextColumn("물품명"),
                                "보유수량": st.column_config.ProgressColumn(
                                    "보유 수량",
                                    format="%d개",
                                    min_value=0,
                                    max_value=int(df_grouped["보유수량"].max())
                                )
                            })
            else:
                st.warning("보유 수량 정보가 없습니다.")
        
        with col2_2:
            st.markdown("### 🏬 센터별 보유 현황")
            selected_prefer_item = st.selectbox("선호 물품 선택", list(preferCnttgClscd.values()), key="col2_prefer")
            selected_prefer_code = [k for k, v in preferCnttgClscd.items() if v == selected_prefer_item][0]

            @st.cache_data
            def load_prefer_by_item(area_cd, unity_signgu_cd, prefer_code):
                df = utils.getPreferInfo(prefer_cnttg_clscd=prefer_code, area_cd=area_cd, unity_signgu_cd=unity_signgu_cd)
                df["센터명"] = df["spctrCd"].map(spctrCd)
                df["보유수량"] = df["holdQy"].astype(float)
                df["lat"] = df["lat"].astype(float)
                df["lon"] = df["lng"].astype(float)
                return df.dropna(subset=["센터명", "lat", "lon"])

            df_item = load_prefer_by_item(area_cd=selected_sido_code, unity_signgu_cd=selected_sigungu_code, prefer_code=selected_prefer_code)

            if not df_item.empty:
                import pydeck as pdk
                chart_data = df_item.copy()
                hex_layer = pdk.Layer(
                    "HexagonLayer",
                    data=chart_data,
                    get_position="[lon, lat]",
                    radius=200,
                    elevation_scale=4,
                    elevation_range=[0, 1000],
                    pickable=True,
                    extruded=True
                )
                scatter_layer = pdk.Layer(
                    "ScatterplotLayer",
                    data=chart_data,
                    get_position="[lon, lat]",
                    get_color="[200, 30, 0, 160]",
                    get_radius=200
                )
                view_state = pdk.ViewState(
                    latitude=chart_data["lat"].mean(),
                    longitude=chart_data["lon"].mean(),
                    zoom=11,
                    pitch=50
                )
                st.pydeck_chart(pdk.Deck(
                    map_style=None,
                    initial_view_state=view_state,
                    layers=[hex_layer, scatter_layer],
                    tooltip={"text": "{센터명}\n보유수량: {보유수량}개"}
                ))
            else:
                st.info("해당 물품을 보유한 센터 정보가 없습니다.")


# ------------------------
# 🏆 Tab 2: 시군구 내 Top 5 센터
# ------------------------
with tab2:
    col1, col2, col3 = st.columns(3)
    with col1:
        selected_sido2 = st.selectbox("시도 선택 (Tab2)", list(areaCd.values()), key="tab2_sido")
        selected_sido_code2 = [k for k, v in areaCd.items() if v == selected_sido2][0]

    with col2:
        sigungu_options2 = {
            k: v for k, v in unitySignguCd.items() if k.startswith(selected_sido_code2)
        }
        selected_sigungu_name2 = st.selectbox("지역구 선택", list(sigungu_options2.values()), key="tab2_sigungu")
        selected_sigungu_code2 = [k for k, v in sigungu_options2.items() if v == selected_sigungu_name2][0]

    with col3:
        selected_prefer_name2 = st.selectbox("선호물품 선택", list(preferCnttgClscd.values()), key="tab2_prefer")
        selected_prefer_code2 = [k for k, v in preferCnttgClscd.items() if v == selected_prefer_name2][0]

    @st.cache_data
    def load_prefer_data(prefer_code):
        df = utils.getPreferInfo(prefer_cnttg_clscd=prefer_code)
        df["시도"] = df["areaCd"].map(areaCd)
        df["시군구"] = df["unitySignguCd"].map(unitySignguCd)
        df["센터명"] = df["spctrCd"].map(spctrCd)
        df["보유수량"] = df["holdQy"]
        return df.dropna(subset=["센터명", "시도", "시군구"])

    df2 = load_prefer_data(selected_prefer_code2)

    filtered = df2[
        (df2["시도"] == selected_sido2) &
        (df2["시군구"] == selected_sigungu_name2)
    ]

    df_top5 = filtered.sort_values("보유수량", ascending=False).head(5)

    if not df_top5.empty:
        st.markdown(f"### 📦 {selected_sido2} {selected_sigungu_name2} 지역 `{selected_prefer_name2}` 보유량 Top 5 센터")
        fig, ax = plt.subplots(figsize=(8, 5))
        sns.barplot(data=df_top5, x="센터명", y="보유수량", palette="YlOrBr", ax=ax)
        ax.set_title(f"Top 5 센터별 `{selected_prefer_name2}` 보유 수량")
        plt.xticks(rotation=30, ha="right")
        st.pyplot(fig)
    else:
        st.warning("해당 지역에는 해당 물품의 보유 정보가 없습니다.")


In [ ]:
import streamlit as st
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
from geopy.distance import geodesic
import streamlit.components.v1 as components
from var import areaCd, unitySignguCd, spctrCd, spctrSecd as spctrSecd_dict
import utils

st.set_page_config(layout="wide")

# 한글 폰트 설정 (맑은 고딕)
font_path = "C:/Windows/Fonts/malgun.ttf"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()
plt.style.use('dark_background')

centers_df = pd.read_csv("./data/food_centers_with_region_and_coords.csv", encoding='utf-8', dtype=object)
centers_df = centers_df.dropna(subset=["위도", "경도"])

sido_code_name = areaCd
sido_name_code = {v: k for k, v in sido_code_name.items()}
sigungu_code_name = unitySignguCd
sigungu_name_code = {v: k for k, v in sigungu_code_name.items()}
spctrCd_code_name = spctrCd
spctrCd_name_code = {v: k for k, v in spctrCd_code_name.items()}

if "selected_sido" not in st.session_state:
    st.session_state.selected_sido = "선택 안 함"
if "reset_requested" not in st.session_state:
    st.session_state.reset_requested = False

st.sidebar.subheader("📍 지역 선택")

if st.sidebar.button("🔄 다시 선택하기"):
    st.session_state.reset_requested = True
    st.rerun()

if st.session_state.reset_requested:
    st.session_state.selected_sido = "선택 안 함"
    st.session_state.reset_requested = False

sido_options = ["선택 안 함"] + list(sido_name_code.keys())
default_sido = st.session_state.selected_sido if st.session_state.selected_sido in sido_options else "선택 안 함"
st.session_state.selected_sido = st.sidebar.selectbox("시도", options=sido_options, index=sido_options.index(default_sido))

filtered_df = centers_df.copy()
if st.session_state.selected_sido != "선택 안 함":
    filtered_df = filtered_df[filtered_df["areaCd"] == sido_name_code[st.session_state.selected_sido]]

food_centers = [
    {
        "label": row["spctrCd"],
        "name": spctrCd_code_name.get(row["spctrCd"], row["spctrCd"]),
        "address": row["spctrAdres"],
        "phone": row["spctrTelno"],
        "lat": row["위도"],
        "lon": row["경도"],
        "sido": sido_code_name.get(row["areaCd"], row["areaCd"]),
        "sigungu": sigungu_code_name.get(row["unitySignguCd"], row["unitySignguCd"]),
        "type": spctrSecd_dict.get(row["spctrSecd"], row["spctrSecd"])
    }
    for _, row in filtered_df.iterrows()
]

left_col, right_col = st.columns([1, 2])

with left_col:
    html_string = f"""
    <!DOCTYPE html>
    <html>
    <head>
      <meta charset=\"utf-8\" />
      <title>푸드뱅크 지도</title>
      <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">
      <style>
        html, body {{ margin: 0; padding: 0; height: 100%; font-family: Arial, sans-serif; }}
        #map {{ height: 650px; border-radius: 12px; box-shadow: 0 0 15px rgba(0,0,0,0.15); }}
      </style>
      <link rel=\"stylesheet\" href=\"https://unpkg.com/leaflet@1.9.3/dist/leaflet.css\" />
      <script src=\"https://unpkg.com/leaflet@1.9.3/dist/leaflet.js\"></script>
    </head>
    <body>
    <div id=\"map\"></div>
    <script>
      const map = L.map('map').setView([36.5, 127.8], 7);
      L.tileLayer('https://{{s}}.basemaps.cartocdn.com/dark_all/{{z}}/{{x}}/{{y}}.png', {{
        attribution: '&copy; OpenStreetMap & CartoDB'
      }}).addTo(map);

      const centers = {json.dumps(food_centers)};
      centers.forEach(c => {{
        const popupHtml = `<b>${{c.name}}</b><br/>주소: ${{c.address}}<br/>전화: ${{c.phone}}<br/><i>${{c.sido}} ${{c.sigungu}}</i><br/><span style='color:gray;'>[${{c.type}}]</span>`;
        let color = "#007BFF";
        if (c.type === "푸드마켓") color = "#28a745";
        else if (c.type === "물류센터") color = "#fd7e14";

        L.circleMarker([c.lat, c.lon], {{
          radius: 6,
          color: color,
          fillColor: color,
          fillOpacity: 0.9
        }}).bindPopup(popupHtml).addTo(map);
      }});

      if (centers.length > 0) {{
        const group = new L.featureGroup(centers.map(c => L.marker([c.lat, c.lon])));
        map.fitBounds(group.getBounds());
      }}
    </script>
    </body>
    </html>
    """
    components.html(html_string, height=650)

with right_col:
    if st.session_state.selected_sido != "선택 안 함":
        st.markdown(f"### 📈 {st.session_state.selected_sido} 지역 통계 요약")
        try:
            areaCd = sido_name_code[st.session_state.selected_sido]
            months = pd.date_range(start="2023-01-01", end="2024-12-01", freq="3MS").strftime("%Y%m").tolist()

            user_records, provd_records, rcept_records = [], [], []

            for ym in months:
                user = utils.getUserInfo(area_cd=areaCd, stdr_ym=ym)
                provd = utils.getProvdStat(area_cd=areaCd, stdr_ym=ym)
                rcept = utils.getRceptStat(area_cd=areaCd, stdr_ym=ym)

                if not user.empty:
                    user_records.append({"월": ym, "이용자 수": user['userCo'].sum(), "이용 횟수": user['useCo'].sum()})
                if not provd.empty:
                    provd_records.append({"월": ym, "제공 건수": provd['provdCo'].sum(), "제공 금액": provd['provdAmt'].sum()})
                if not rcept.empty:
                    rcept_records.append({"월": ym, "접수 금액": rcept['rceptAmt'].sum()})

            df_user = pd.DataFrame(user_records)
            df_provd = pd.DataFrame(provd_records)
            df_rcept = pd.DataFrame(rcept_records)

            def dual_axis_plot(df, col1, col2, title):
                fig, ax1 = plt.subplots(figsize=(5.5, 2.5))
                ax2 = ax1.twinx()
                sns.lineplot(data=df, x="월", y=col1, ax=ax1, color="skyblue", label=col1)
                sns.lineplot(data=df, x="월", y=col2, ax=ax2, color="orange", label=col2)
                ax1.set_ylabel(col1)
                ax2.set_ylabel(col2)
                ax1.tick_params(axis='x', rotation=45)
                ax1.legend(loc='upper left')
                ax2.legend(loc='upper right')
                st.markdown(f"#### {title}")
                st.pyplot(fig)

            plot_cols = st.columns(2)
            with plot_cols[0]:
                dual_axis_plot(df_user, "이용자 수", "이용 횟수", "📊 이용자 통계")
            with plot_cols[1]:
                dual_axis_plot(df_provd, "제공 건수", "제공 금액", "📊 제공 통계")
            with plot_cols[0]:
                if not df_rcept.empty:
                    fig, ax = plt.subplots(figsize=(5.5, 2.5))
                    sns.lineplot(data=df_rcept, x="월", y="접수 금액", ax=ax, color="lightgreen")
                    ax.tick_params(axis='x', rotation=45)
                    st.markdown("#### 📊 접수 통계")
                    st.pyplot(fig)

        except Exception as e:
            st.warning(f"통계 데이터를 불러오지 못했습니다: {e}")

st.markdown("✅ 마커를 클릭하면 센터 정보를 확인할 수 있습니다.")


In [1]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_excel('./data/foodbank.xlsx', header=None)

In [26]:
dict = {}

for row in df.iterrows():
    dict[row[1][0]] = row[1][1]

In [27]:
dict

{'S00101': '전국푸드뱅크',
 'S00201': '서울시광역푸드뱅크',
 'S00301': '영등포사랑나눔푸드뱅크',
 'S00401': '성동희망푸드뱅크',
 'S00501': '종로구푸드뱅크',
 'S00601': '동대문구푸드뱅크',
 'S00701': '중랑구푸드뱅크',
 'S00801': '강북구푸드뱅크',
 'S00901': '도봉구푸드뱅크',
 'S01001': '노원구월계푸드뱅크',
 'S01101': '노원구평화푸드뱅크',
 'S01201': '은평구푸드뱅크',
 'S01301': '강서구푸드뱅크',
 'S01401': '구로희망푸드뱅크',
 'S01501': '금천구푸드뱅크',
 'S01601': '양천구푸드뱅크',
 'S01701': '관악구푸드뱅크',
 'S01801': '서초구푸드뱅크',
 'S01901': '송파구푸드뱅크',
 'S02001': '강동구기초푸드뱅크',
 'S02101': '성북구푸드뱅크',
 'S02201': '동작기초푸드뱅크',
 'S02301': '마포행복나눔기초푸드뱅크',
 'S02401': '용산구푸드뱅크',
 'S02501': '부산시광역푸드뱅크',
 'S02601': '중구푸드뱅크',
 'S02701': '서구푸드뱅크',
 'S02801': '동구푸드뱅크',
 'S02901': '영도구푸드뱅크',
 'S03001': '부산진구푸드뱅크',
 'S03101': '동래구푸드뱅크',
 'S03201': '남구푸드뱅크',
 'S03301': '북구푸드뱅크',
 'S03401': '해운대기초푸드뱅크',
 'S03501': '사하구푸드뱅크',
 'S03601': '금정구푸드뱅크',
 'S03701': '강서구푸드뱅크',
 'S03801': '연제구푸드뱅크',
 'S03901': '수영구푸드뱅크',
 'S04001': '사상구푸드뱅크',
 'S04101': '기장군푸드뱅크',
 'S04201': '대구시광역푸드뱅크',
 'S04301': '달서구본동푸드뱅크',
 'S04401': '달서구성서푸드뱅크',
 'S045

# 지도 렌더링
html_string = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8" />
  <title>푸드뱅크 지도</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    html, body {{ margin: 0; padding: 0; height: 100%; font-family: Arial, sans-serif; }}
    #map {{ height: 100%; border-radius: 12px; box-shadow: 0 0 15px rgba(0,0,0,0.15); }}
  </style>
  <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.3/dist/leaflet.css" />
  <script src="https://unpkg.com/leaflet@1.9.3/dist/leaflet.js"></script>
</head>
<body>
<div id="map"></div>
<script>
  const map = L.map('map').setView([36.5, 127.8], 7);
  L.tileLayer('https://{{s}}.basemaps.cartocdn.com/light_all/{{z}}/{{x}}/{{y}}.png', {{
    attribution: '&copy; OpenStreetMap & CartoDB'
  }}).addTo(map);

  const centers = {json.dumps(food_centers)};
  centers.forEach(c => {{
    const popupHtml = `<b>${{c.name}}</b><br/>주소: ${{c.address}}<br/>전화: ${{c.phone}}<br/><i>${{c.sido}} ${{c.sigungu}}</i><br/><span style='color:gray;'>[${{c.type}}]</span>`;
    let color = "#007BFF";
    if (c.type === "푸드마켓") color = "#28a745";
    else if (c.type === "물류센터") color = "#fd7e14";

    const marker = L.circleMarker([c.lat, c.lon], {{
      radius: 6,
      color: color,
      fillColor: color,
      fillOpacity: 0.9
    }}).bindPopup(popupHtml).addTo(map);

    marker.on('click', () => {{
      const payload = {{
        type: "centerSelect",
        value: {{
          sido: c.sido,
          sigungu: c.sigungu,
          label: c.label
        }}
      }};
      console.log("📤 메시지 전송됨", payload);
      window.parent.postMessage(payload, "*");
    }});
  }});

  if (centers.length > 0) {{
    const group = new L.featureGroup(centers.map(c => L.marker([c.lat, c.lon])));
    map.fitBounds(group.getBounds());
  }}
</script>
</body>
</html>
"""

In [ ]:
      Streamlit.setComponentValue(payload);


In [3]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("./data/food_centers_with_region_and_coords.csv", encoding='utf-8')

# areaCd 컬럼이 존재하는지 확인 후 처리
if "areaCd" in df.columns:
    # 문자열로 변환하고, 길이가 1인 경우 앞에 '0'을 붙임
    df["areaCd"] = df["areaCd"].astype(str).str.zfill(2)
    
# unitySignguCd 컬럼이 존재하는지 확인 후 처리
if "unitySignguCd" in df.columns:
    # 문자열로 변환하고, 길이가 7인 경우 앞에 '0'을 붙임
    df["unitySignguCd"] = df["unitySignguCd"].astype(str).str.zfill(8)

    # 필요한 경우 저장
    df.to_csv("./data/food_centers_with_region_and_coords.csv", index=False, encoding='utf-8')
else:
    print("unitySignguCd 컬럼이 데이터프레임에 존재하지 않습니다.")

In [10]:
df = pd.read_csv("./data/food_centers_with_region.csv", encoding="UTF-8")

In [ ]:
import pandas as pd
import requests
import time
from tqdm import tqdm

df = pd.read_csv("./data/food_centers_with_region.csv", encoding="UTF-8")
KAKAO_REST_API_KEY = "7758866813611c4b671d0ed485e11ce3"

def geocode(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"}
    res = requests.get(url, headers=headers, params={"query": address})
    if res.status_code == 200 and res.json()["documents"]:
        loc = res.json()["documents"][0]
        return loc["y"], loc["x"]
    return None, None

coords = []
for _, row in tqdm(df.iterrows()):
    addr = row["spctrAdres"] if pd.notnull(row["spctrAdres"]) else row["spctrDetailAdres"]
    lat, lon = geocode(addr)
    coords.append((row["spctrCd"], lat, lon))
    time.sleep(0.1)

geo_df = pd.DataFrame(coords, columns=["spctrCd", "위도", "경도"])
# 


437it [01:20,  5.46it/s]


In [11]:
df = pd.merge(df, geo_df, on='spctrCd')
df.to_csv("./data/food_centers_with_region_and_coords.csv", index=False, encoding="utf-8-sig")

# **Test**

In [ ]:
# Re-import necessary modules due to code execution state reset
import requests
import xml.etree.ElementTree as ET
import json
from tqdm import tqdm
import time

# Load the center name-to-code dictionary again
with open("./data/center_name_to_code.json", "r", encoding="utf-8") as f:
    name_to_code = json.load(f)
    
# Define the function to query the API
def getSpctrInfo(page_no=1, num_of_rows=2, data_type='xml', stdr_ym='202412', area_cd='', unity_signgu_cd='', spctr_uncd='', spctr_secd='', spctr_cd='', spctr_stscd='', oper_mby_lclas_cd='', oper_mby_sclas_cd='', sttemnt_seccd='', regist_de=''):
    url = "http://apis.data.go.kr/B460014/foodBankInfoService2/getSpctrInfo"
    params = {
        "serviceKey": "UHD0d7Tof2U4mU3gGH+3QxTtjc5FwFNnHyqyTBWq+nI5TeubdJF6ytylohZXVLp+7oVsmoG6CUVmNaqKsyRGIA==",
        "pageNo": page_no,
        "numOfRows": num_of_rows,
        "dataType": data_type,
        "stdrYm": stdr_ym,
        "areaCd": area_cd,
        "unitySignguCd": unity_signgu_cd,
        "spctrUncd": spctr_uncd,
        "spctrSecd": spctr_secd,
        "spctrCd": spctr_cd,
        "spctrStscd": spctr_stscd,
        "operMbyLclasCd": oper_mby_lclas_cd,
        "operMbySclasCd": oper_mby_sclas_cd,
        "sttemntSeccd": sttemnt_seccd,
        "registDe": regist_de
    }
    try:
        response = requests.get(url, params=params)
        root = ET.fromstring(response.content)
        print(root)
        return len(root.findall(".//item")) > 0
    except Exception as e:
        return False

# Iterate through all codes and identify non-working ones (limiting to 100 for demonstration)
unavailable_centers = []
for name, code in tqdm(list(name_to_code.items())):  # limit for testing
    if not getSpctrInfo(spctr_cd=code):
        print(code)
        unavailable_centers.append(name)
        time.sleep(2)

unavailable_centers  # Show a preview


  0%|          | 0/628 [00:00<?, ?it/s]

100%|██████████| 628/628 [04:23<00:00,  2.38it/s]


['영등포사랑나눔푸드뱅크',
 '성동희망푸드뱅크',
 '종로구푸드뱅크',
 '동대문구푸드뱅크',
 '중랑구푸드뱅크',
 '강북구푸드뱅크',
 '도봉구푸드뱅크',
 '노원구월계푸드뱅크',
 '노원구평화푸드뱅크',
 '은평구푸드뱅크',
 '구로희망푸드뱅크',
 '금천구푸드뱅크',
 '양천구푸드뱅크',
 '관악구푸드뱅크',
 '서초구푸드뱅크',
 '송파구푸드뱅크',
 '강동구기초푸드뱅크',
 '성북구푸드뱅크',
 '동작기초푸드뱅크',
 '마포행복나눔기초푸드뱅크',
 '용산구푸드뱅크',
 '중구푸드뱅크',
 '남구푸드뱅크',
 '사상구푸드뱅크',
 '동구베다니푸드뱅크',
 '북구선린푸드뱅크',
 '수성구청곡푸드뱅크',
 '수성구애활원푸드뱅크',
 '동구기초푸드뱅크',
 '대전서구기초푸드뱅크',
 '유성구푸드뱅크',
 '대덕구푸드뱅크',
 '울주푸드뱅크',
 '성남기초푸드뱅크',
 '성남시하사함푸드뱅크',
 '의정부시푸드뱅크',
 '광명시푸드뱅크',
 '안산샬롬푸드뱅크',
 '고양시해냄푸드뱅크',
 '고양시흰돌기초푸드뱅크',
 '고양시문촌9푸드뱅크',
 '고양시열두푸드뱅크',
 '과천시푸드뱅크',
 '시흥시목감푸드뱅크',
 '김포(기초)푸드뱅크',
 '광주시푸드뱅크',
 '양평군푸드뱅크',
 '남양주시푸드뱅크',
 '원주시푸드뱅크',
 '청주시상당푸드뱅크',
 '천안기초푸드뱅크',
 '홍성군기초푸드뱅크',
 '익산시기초푸드뱅크',
 '임실군기초푸드뱅크',
 '담양군푸드뱅크',
 '구례군푸드뱅크',
 '장흥군푸드뱅크',
 '해남군푸드뱅크',
 '완도군푸드뱅크',
 '신안군푸드뱅크',
 '고령군푸드뱅크',
 '영덕군푸드뱅크',
 '포항시모자원푸드뱅크',
 '의령군푸드뱅크',
 '거창군푸드뱅크',
 '광진구푸드뱅크',
 '중구드림하티푸드뱅크',
 '성남시수정푸드뱅크',
 '영동군푸드뱅크',
 '성남시빈첸시오푸드뱅크',
 '용인시푸드뱅크',
 '강남구기초푸드뱅크',
 '예산군푸드뱅크',
 '연기군푸드뱅크',
 '부안군기초푸드뱅크',
 '부여군푸드뱅크',
 '예천군푸드뱅크',


In [54]:
for name in unavailable_centers:
    name_to_code.pop(name)

In [57]:
len(name_to_code)

403

In [62]:
json_string = json.dumps(name_to_code, indent=4, ensure_ascii = False)

In [ ]:
with open("./data/center_name_to_code.json", "w", encoding='UTF-8-sig') as f:
    f.write(json_string)

In [67]:
# Define the function to query the API
def getSpctrInfo(page_no=1, num_of_rows=500, data_type='xml', stdr_ym='202412', area_cd='', unity_signgu_cd='', spctr_uncd='', spctr_secd='', spctr_cd='', spctr_stscd='', oper_mby_lclas_cd='', oper_mby_sclas_cd='', sttemnt_seccd='', regist_de=''):
    url = "http://apis.data.go.kr/B460014/foodBankInfoService2/getSpctrInfo"
    params = {
        "serviceKey": "UHD0d7Tof2U4mU3gGH+3QxTtjc5FwFNnHyqyTBWq+nI5TeubdJF6ytylohZXVLp+7oVsmoG6CUVmNaqKsyRGIA==",
        "pageNo": page_no,
        "numOfRows": num_of_rows,
        "dataType": data_type,
        "stdrYm": stdr_ym,
        "areaCd": area_cd,
        "unitySignguCd": unity_signgu_cd,
        "spctrUncd": spctr_uncd,
        "spctrSecd": spctr_secd,
        "spctrCd": spctr_cd,
        "spctrStscd": spctr_stscd,
        "operMbyLclasCd": oper_mby_lclas_cd,
        "operMbySclasCd": oper_mby_sclas_cd,
        "sttemntSeccd": sttemnt_seccd,
        "registDe": regist_de
    }
    try:
        response = requests.get(url, params=params)
        root = ET.fromstring(response.content)
        return pd.DataFrame([{
            "stdrYm": i.findtext("stdrYm"),
            "areaCd": i.findtext("areaCd"),
            "unitySignguCd": i.findtext("unitySignguCd"),
            "spctrUncd": i.findtext("spctrUncd"),
            "spctrSecd": i.findtext("spctrSecd"),
            "spctrCd": i.findtext("spctrCd"),
            "spctrTelno": i.findtext("spctrTelno"),
            "spctrAdres": i.findtext("spctrAdres"),
            "spctrDetailAdres": i.findtext("spctrDetailAdres"),
            "spctrStscd": i.findtext("spctrStscd"),
            "operMbyLclasCd": i.findtext("operMbyLclasCd"),
            "operMbySclasCd": i.findtext("operMbySclasCd"),
            "sttemntSeccd": i.findtext("sttemntSeccd"),
            "registDe": i.findtext("registDe"),
            "rceptAmt": float(i.findtext("rceptAmt", 0)),
            "provdAmt": float(i.findtext("provdAmt", 0)),
            "undtakeAmt": float(i.findtext("undtakeAmt", 0)),
            "trnsferAmt": float(i.findtext("trnsferAmt", 0)),
            "userCo": int(i.findtext("userCo", 0))
        } for i in root.findall(".//item")])
    except Exception as e:
        return False
    
# Re-run API collection after loading corrected dictionary
full_df = getSpctrInfo()


In [77]:
full_df

,stdrYm,areaCd,unitySignguCd,spctrUncd,spctrSecd,spctrCd,spctrTelno,spctrAdres,spctrDetailAdres,spctrStscd,...,operMbySclasCd,sttemntSeccd,registDe,rceptAmt,provdAmt,undtakeAmt,trnsferAmt,userCo,시도명,지역구
0,202412,01,02000014,01,20,S00101,02-2077-3985,서울특별시 마포구 만리재로 14,한국사회복지회관 1201호,1,...,0101,02,20020520,2.679328e+09,5.600000e+06,0.0,2.871599e+09,2,서울,마포구
1,202412,02,02000004,02,20,S00201,02-905-1377,서울특별시 성동구 자동차시장3길 64,"(용답동, 중랑물재생센터)",1,...,0101,02,20001205,6.420755e+08,2.405558e+07,4410000.0,5.232800e+08,417,서울,성동구
2,202412,03,03000006,02,20,S02501,051-791-1377,부산광역시 동래구 낙민로 25,(낙민동)부산사회복지종합센터 302호,1,...,0101,02,20140701,4.299639e+08,1.453206e+08,428016010.0,7.359790e+08,69,부산,동래구
3,202412,03,03000001,03,20,S02601,051-464-3137,부산 중구 대청동4가,75-7,1,...,0101,01,20020101,0.000000e+00,0.000000e+00,18001109.0,0.000000e+00,1,부산,중구
4,202412,03,03000002,03,20,S02701,051-253-1922,부산광역시 서구 망양로193번길 104,(동대신동1가),1,...,0101,01,20020101,1.373020e+06,5.081402e+06,16357891.0,0.000000e+00,6,부산,서구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,202412,11,11000800,03,20,S72701,043-422-1299,충청북도 단양군 단양읍 수변로 83,3층,1,...,0101,01,20221201,0.000000e+00,0.000000e+00,9736649.0,0.000000e+00,1,충북,단양군
433,202412,05,05000005,03,20,S72801,032-437-1377,인천광역시 남동구 경인로 744,105호 (간석동),1,...,0103,02,20221206,3.687726e+07,7.486890e+07,38569324.0,2.836336e+06,151,인천,남동구
434,202412,14,14000300,03,20,S72901,061-781-7970,전라남도 구례군 구례읍 구례로 508,구례종합사회복지관 1층,1,...,0302,01,20221227,2.825450e+06,3.220260e+06,12406848.0,0.000000e+00,102,전남,구례군
435,202412,14,14001500,03,20,S73001,061-554-1377,전라남도 완도군 완도읍 장보고대로 216,,1,...,0103,01,20231102,0.000000e+00,2.131210e+07,15876096.0,0.000000e+00,80,전남,완도군


In [71]:
full_df.to_csv('./data/food_centers_api.csv', index=False, encoding='UTF-8-sig')

In [75]:
import requests
import pandas as pd
from tqdm import tqdm

# 카카오 REST API 키 설정 (여기에 본인의 키를 넣어주세요)
KAKAO_REST_API_KEY = "7758866813611c4b671d0ed485e11ce3"

def get_region_from_address(address, api_key=KAKAO_REST_API_KEY):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}

    try:
        response = requests.get(url, headers=headers, params=params, timeout=5)
        result = response.json()
        if result['documents']:
            doc = result['documents'][0]['address']
            return doc['region_1depth_name'], doc['region_2depth_name']
        else:
            return None, None
    except Exception as e:
        return None, None


In [76]:
# 예시: 주소 컬럼이 'spctrAdres'인 데이터프레임
full_df['시도명'], full_df['지역구'] = zip(*full_df['spctrAdres'].map(lambda x: get_region_from_address(x)))
full_df.to_csv("food_centers_with_region.csv", index=False, encoding="utf-8-sig")

# **Test**

In [13]:
df = pd.read_excel('./data/code_center.xlsx', header=None)
df2 = pd.read_csv('./data/5_`24년 센터정보.csv', encoding="cp949")

In [14]:
print(df.shape[0], df2.shape[0])

650 437


In [43]:
set(df2['지원센터명']).difference(df[1])

{'강남푸드지원센터',
 '강북잇다푸드뱅크마켓센터',
 '강진군기초푸드뱅크',
 '고성푸드뱅크사랑나눔센터',
 '곡성지역자활센터 푸드뱅크',
 '광명시푸드뱅크마켓센터',
 '구례군 다빈푸드뱅크',
 '나눔과기쁨 창원푸드뱅크',
 '나주시기초푸드뱅크',
 '남동구새희망푸드뱅크',
 '남원시사회복지협의회기초푸드뱅크',
 '두암기초푸드뱅크',
 '무주행복나눔푸드마켓·뱅크',
 '문막푸드뱅크',
 '문수기초푸드뱅크',
 '보령시기초푸드뱅크',
 '보성기초푸드뱅크',
 '봉담아리푸드뱅크\t',
 '부안행복나눔푸드마켓·뱅크',
 '부평구푸드뱅크1호점',
 '북구기초푸드뱅크',
 '사랑나눔기초푸드뱅크',
 '사랑나눔푸드마켓·뱅크',
 '사랑의등불 강서푸드뱅크마켓',
 '산마을푸드뱅크',
 '서구기초푸드뱅크',
 '서구푸드마켓',
 '서대문정담은푸드마켓',
 '서산행복나눔푸드마켓',
 '선산푸드뱅크',
 '섬기고나눔기초푸드뱅크',
 '성남열린푸드뱅크·마켓',
 '송파푸드마켓  송파푸드뱅크',
 '순천시푸드마켓',
 '시흥푸드뱅크마켓센터',
 '쌍봉기초푸드뱅크',
 '쌍촌기초푸드뱅크',
 '안산나눔푸드뱅크',
 '안산푸드뱅크',
 '안양착한푸드마켓',
 '연화마을푸드뱅크',
 '예산군기초푸드뱅크',
 '오치기초푸드뱅크',
 '옥천군행복나눔푸드뱅크마켓',
 '완도군기초푸드뱅크',
 '원주만나푸드뱅크',
 '의정부푸드뱅크마켓',
 '장성군 기초푸드뱅크',
 '전주글로벌푸드뱅크',
 '전주시중앙푸드마켓',
 '증평군기초푸드뱅크',
 '청송군푸드뱅크',
 '통영푸드뱅크',
 '함평기초푸드뱅크',
 '행복나눔기초푸드뱅크',
 '행복마당 푸드마켓',
 '홍성군푸드마켓뱅크',
 '희망나눔푸드뱅크',
 '흰돌푸드뱅크마켓센터'}

In [17]:
df[1]

0            전국푸드뱅크
1         서울시광역푸드뱅크
2       영등포사랑나눔푸드뱅크
3          성동희망푸드뱅크
4           종로구푸드뱅크
           ...     
645         새희망푸드뱅크
646         참좋은푸드마켓
647         동군산푸드뱅크
648       전주시새샘푸드뱅크
649    함평희망나눔기초푸드뱅크
Name: 1, Length: 650, dtype: object

In [9]:
df2.head()

,번호,지역,시군구,지원센터단위,지원센터구분,지원센터명,센터전화번호,주소,상세주소,기타유의사항
0,1,중앙,서울특별시 마포구,전국,푸드뱅크,전국푸드뱅크,02-2077-3985,서울특별시 마포구 만리재로 14,한국사회복지회관 1201호,NaN
1,2,강원,강원도 강릉시,광역,푸드뱅크,강원도광역푸드뱅크,033-646-0197,강원도 강릉시 구정면 구정중앙로 158-10,NaN,NaN
2,5,경기,경기도 화성시,광역,푸드뱅크,경기도광역푸드뱅크,031-294-1377,경기도 화성시 정남면 괘랑1길 42-30,A동,NaN
3,6,경남,경상남도 창원시 마산회원구,광역,푸드뱅크,경남광역푸드뱅크,070-4266-1202,경상남도 김해시 칠산로 128,(신문동) 경상남도광역기부식품등지원센터,NaN
4,7,경북,경상북도 안동시,광역,푸드뱅크,경북광역푸드뱅크,054-843-8550,경상북도 안동시 광명로 159,3층 302호 (옥동),NaN


In [28]:
pd.merge(df, df2, how='inner', left_on=1, right_on='지원센터명')

,0,1,번호,지역,시군구,지원센터단위,지원센터구분,지원센터명,센터전화번호,주소,상세주소,기타유의사항
0,S00101,전국푸드뱅크,1,중앙,서울특별시 마포구,전국,푸드뱅크,전국푸드뱅크,02-2077-3985,서울특별시 마포구 만리재로 14,한국사회복지회관 1201호,NaN
1,S00201,서울시광역푸드뱅크,12,서울,서울특별시 성동구,광역,푸드뱅크,서울시광역푸드뱅크,02-905-1377,서울특별시 성동구 자동차시장3길 64,(용답동 중랑물재생센터),NaN
2,S01301,강서구푸드뱅크,30,부산,부산광역시 강서구,기초,푸드뱅크,강서구푸드뱅크,051-271-0561,부산 강서구 명지동,627-28,NaN
3,S02501,부산시광역푸드뱅크,11,부산,부산광역시 동래구,광역,푸드뱅크,부산시광역푸드뱅크,051-791-1377,부산광역시 동래구 낙민로 25,(낙민동)부산사회복지종합센터 302호,NaN
4,S02601,중구푸드뱅크,350,인천,인천광역시 중구,기초,푸드뱅크,중구푸드뱅크,032-761-1399,인천광역시 중구 홍예문로 77,(전동17-38),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
416,S70101,대전행복나눔푸드마켓(유천점),130,대전,대전광역시 중구,기초,푸드마켓,대전행복나눔푸드마켓(유천점),042-522-7100,대전광역시 중구 유천로 93,1층 (유천동),NaN
417,S70401,참좋은푸드마켓,366,전북,전라북도 정읍시,기초,푸드마켓,참좋은푸드마켓,063-531-0191,전라북도 정읍시 명덕로 55,(연지동),NaN
418,S70501,동군산푸드뱅크,143,전북,전라북도 군산시,기초,푸드뱅크,동군산푸드뱅크,063-451-5403,전라북도 군산시 서수면 하금길 37-1,NaN,NaN
419,S70601,전주시새샘푸드뱅크,335,전북,전라북도 전주시 완산구,기초,푸드뱅크,전주시새샘푸드뱅크,063-286-0217,전북특별자치도 전주시 완산구 고덕산1길 139,(대성동),NaN


In [33]:
len(set(df2['지원센터명'])), len(set(pd.merge(df, df2, left_on=1, right_on='지원센터명', how='inner')['지원센터명']))

(428, 369)

In [36]:
len(set(df[1])), len(set(df2['지원센터명']))

(629, 428)

In [42]:
pd.merge(df, df2, left_on=1, right_on='지원센터명', how='inner').drop_duplicates(subset=[1, '지원센터명'])



,0,1,번호,지역,시군구,지원센터단위,지원센터구분,지원센터명,센터전화번호,주소,상세주소,기타유의사항
0,S00101,전국푸드뱅크,1,중앙,서울특별시 마포구,전국,푸드뱅크,전국푸드뱅크,02-2077-3985,서울특별시 마포구 만리재로 14,한국사회복지회관 1201호,NaN
1,S00201,서울시광역푸드뱅크,12,서울,서울특별시 성동구,광역,푸드뱅크,서울시광역푸드뱅크,02-905-1377,서울특별시 성동구 자동차시장3길 64,(용답동 중랑물재생센터),NaN
2,S01301,강서구푸드뱅크,30,부산,부산광역시 강서구,기초,푸드뱅크,강서구푸드뱅크,051-271-0561,부산 강서구 명지동,627-28,NaN
3,S02501,부산시광역푸드뱅크,11,부산,부산광역시 동래구,광역,푸드뱅크,부산시광역푸드뱅크,051-791-1377,부산광역시 동래구 낙민로 25,(낙민동)부산사회복지종합센터 302호,NaN
4,S02601,중구푸드뱅크,350,인천,인천광역시 중구,기초,푸드뱅크,중구푸드뱅크,032-761-1399,인천광역시 중구 홍예문로 77,(전동17-38),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
416,S70101,대전행복나눔푸드마켓(유천점),130,대전,대전광역시 중구,기초,푸드마켓,대전행복나눔푸드마켓(유천점),042-522-7100,대전광역시 중구 유천로 93,1층 (유천동),NaN
417,S70401,참좋은푸드마켓,366,전북,전라북도 정읍시,기초,푸드마켓,참좋은푸드마켓,063-531-0191,전라북도 정읍시 명덕로 55,(연지동),NaN
418,S70501,동군산푸드뱅크,143,전북,전라북도 군산시,기초,푸드뱅크,동군산푸드뱅크,063-451-5403,전라북도 군산시 서수면 하금길 37-1,NaN,NaN
419,S70601,전주시새샘푸드뱅크,335,전북,전라북도 전주시 완산구,기초,푸드뱅크,전주시새샘푸드뱅크,063-286-0217,전북특별자치도 전주시 완산구 고덕산1길 139,(대성동),NaN


In [31]:
set(df2['지원센터명']).difference(set(pd.merge(df, df2, left_on=1, right_on='지원센터명')['지원센터명']))

{'강남푸드지원센터',
 '강북잇다푸드뱅크마켓센터',
 '강진군기초푸드뱅크',
 '고성푸드뱅크사랑나눔센터',
 '곡성지역자활센터 푸드뱅크',
 '광명시푸드뱅크마켓센터',
 '구례군 다빈푸드뱅크',
 '나눔과기쁨 창원푸드뱅크',
 '나주시기초푸드뱅크',
 '남동구새희망푸드뱅크',
 '남원시사회복지협의회기초푸드뱅크',
 '두암기초푸드뱅크',
 '무주행복나눔푸드마켓·뱅크',
 '문막푸드뱅크',
 '문수기초푸드뱅크',
 '보령시기초푸드뱅크',
 '보성기초푸드뱅크',
 '봉담아리푸드뱅크\t',
 '부안행복나눔푸드마켓·뱅크',
 '부평구푸드뱅크1호점',
 '북구기초푸드뱅크',
 '사랑나눔기초푸드뱅크',
 '사랑나눔푸드마켓·뱅크',
 '사랑의등불 강서푸드뱅크마켓',
 '산마을푸드뱅크',
 '서구기초푸드뱅크',
 '서구푸드마켓',
 '서대문정담은푸드마켓',
 '서산행복나눔푸드마켓',
 '선산푸드뱅크',
 '섬기고나눔기초푸드뱅크',
 '성남열린푸드뱅크·마켓',
 '송파푸드마켓  송파푸드뱅크',
 '순천시푸드마켓',
 '시흥푸드뱅크마켓센터',
 '쌍봉기초푸드뱅크',
 '쌍촌기초푸드뱅크',
 '안산나눔푸드뱅크',
 '안산푸드뱅크',
 '안양착한푸드마켓',
 '연화마을푸드뱅크',
 '예산군기초푸드뱅크',
 '오치기초푸드뱅크',
 '옥천군행복나눔푸드뱅크마켓',
 '완도군기초푸드뱅크',
 '원주만나푸드뱅크',
 '의정부푸드뱅크마켓',
 '장성군 기초푸드뱅크',
 '전주글로벌푸드뱅크',
 '전주시중앙푸드마켓',
 '증평군기초푸드뱅크',
 '청송군푸드뱅크',
 '통영푸드뱅크',
 '함평기초푸드뱅크',
 '행복나눔기초푸드뱅크',
 '행복마당 푸드마켓',
 '홍성군푸드마켓뱅크',
 '희망나눔푸드뱅크',
 '흰돌푸드뱅크마켓센터'}

In [2]:
import pandas as pd

# Load Excel file
df = pd.read_excel('center.xlsx', dtype=str)

# Drop any completely empty rows
df.dropna(how='all', inplace=True)

# Flatten the DataFrame into a dictionary
center_dict = {}
for i in range(0, len(df.columns), 2):
    code_col = df.columns[i]
    name_col = df.columns[i + 1]
    pairs = df[[code_col, name_col]].dropna()
    center_dict.update(dict(zip(pairs[code_col], pairs[name_col])))

# Print part of the dictionary
print(dict(list(center_dict.items())))  # Show first 10 entries


{'S00101': '전국푸드뱅크', 'S00201': '서울시광역푸드뱅크', 'S00301': '영등포사랑나눔푸드뱅크', 'S00401': '성동희망푸드뱅크', 'S00501': '종로구푸드뱅크', 'S00601': '동대문구푸드뱅크', 'S00701': '중랑구푸드뱅크', 'S00801': '강북구푸드뱅크', 'S00901': '도봉구푸드뱅크', 'S01001': '노원구월계푸드뱅크', 'S01101': '노원구평화푸드뱅크', 'S01201': '은평구푸드뱅크', 'S01301': '강서구푸드뱅크', 'S01401': '구로희망푸드뱅크', 'S01501': '금천구푸드뱅크', 'S01601': '양천구푸드뱅크', 'S01701': '관악구푸드뱅크', 'S01801': '서초구푸드뱅크', 'S01901': '송파구푸드뱅크', 'S02001': '강동구기초푸드뱅크', 'S02101': '성북구푸드뱅크', 'S02201': '동작기초푸드뱅크', 'S02301': '마포행복나눔기초푸드뱅크', 'S02401': '용산구푸드뱅크', 'S02501': '부산시광역푸드뱅크', 'S02601': '중구푸드뱅크', 'S02701': '서구푸드뱅크', 'S02801': '동구푸드뱅크', 'S02901': '영도구푸드뱅크', 'S03001': '부산진구푸드뱅크', 'S03101': '동래구푸드뱅크', 'S03201': '남구푸드뱅크', 'S03301': '북구푸드뱅크', 'S03401': '해운대기초푸드뱅크', 'S03501': '사하구푸드뱅크', 'S03601': '금정구푸드뱅크', 'S03701': '강서구푸드뱅크', 'S03801': '연제구푸드뱅크', 'S03901': '수영구푸드뱅크', 'S04001': '사상구푸드뱅크', 'S04101': '기장군푸드뱅크', 'S04201': '대구시광역푸드뱅크', 'S04301': '달서구본동푸드뱅크', 'S04401': '달서구성서푸드뱅크', 'S04501': '동구동촌푸드뱅크', 'S04601': '동구베다니푸드뱅크', 'S04

# **구현 Skeleton Code**

In [3]:
encode = "UHD0d7Tof2U4mU3gGH%2B3QxTtjc5FwFNnHyqyTBWq%2BnI5TeubdJF6ytylohZXVLp%2B7oVsmoG6CUVmNaqKsyRGIA%3D%3D"
decode = "UHD0d7Tof2U4mU3gGH+3QxTtjc5FwFNnHyqyTBWq+nI5TeubdJF6ytylohZXVLp+7oVsmoG6CUVmNaqKsyRGIA=="

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

BASE_URL = "http://apis.data.go.kr/B460014/foodBankInfoService2"
SERVICE_KEY = decode

# 1. 이용자 통계데이터 조회
def getUserInfo(page_no=1, num_of_rows=10, data_type='xml', stdr_ym='202402', area_cd='', unity_signgu_cd='', spctr_cd='', user_seccd='', user_clscd='', happy_trgter_yn=''):
    url = f"{BASE_URL}/getUserInfo"
    params = {
        "serviceKey": SERVICE_KEY,
        "pageNo": page_no,
        "numOfRows": num_of_rows,
        "dataType": data_type,
        "stdrYm": stdr_ym,
        "areaCd": area_cd,
        "unitySignguCd": unity_signgu_cd,
        "spctrCd": spctr_cd,
        "userSeccd": user_seccd,
        "userClscd": user_clscd,
        "happyTrgterYn": happy_trgter_yn
    }
    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)
    return pd.DataFrame([{
        "stdrYm": i.findtext("stdrYm"),
        "areaCd": i.findtext("areaCd"),
        "unitySignguCd": i.findtext("unitySignguCd"),
        "spctrCd": i.findtext("spctrCd"),
        "userSeccd": i.findtext("userSeccd"),
        "userClscd": i.findtext("userClscd"),
        "happyTrgterYn": i.findtext("happyTrgterYn"),
        "useAmt": float(i.findtext("useAmt", 0)),
        "useCo": int(i.findtext("useCo", 0)),
        "userCo": int(i.findtext("userCo", 0))
    } for i in root.findall(".//item")])

# 2. 이용시설단체 통계데이터 조회
def getFcltyGrpInfo(page_no=1, num_of_rows=10, data_type='xml', stdr_ym='202402', area_cd='', unity_signgu_cd='', spctr_cd='', fclty_se_nm='', fclty_grp_clscd=''):
    url = f"{BASE_URL}/getFcltyGrpInfo"
    params = {
        "serviceKey": SERVICE_KEY,
        "pageNo": page_no,
        "numOfRows": num_of_rows,
        "dataType": data_type,
        "stdrYm": stdr_ym,
        "areaCd": area_cd,
        "unitySignguCd": unity_signgu_cd,
        "spctrCd": spctr_cd,
        "fcltySeNm": fclty_se_nm,
        "fcltyGrpClscd": fclty_grp_clscd
    }
    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)
    return pd.DataFrame([{
        "stdrYm": i.findtext("stdrYm"),
        "areaCd": i.findtext("areaCd"),
        "unitySignguCd": i.findtext("unitySignguCd"),
        "spctrCd": i.findtext("spctrCd"),
        "fcltySeNm": i.findtext("fcltySeNm"),
        "fcltyGrpClscd": i.findtext("fcltyGrpClscd"),
        "useAmt": float(i.findtext("useAmt", 0)),
        "useCo": int(i.findtext("useCo", 0)),
        "fcltyGrpCo": int(i.findtext("fcltyGrpCo", 0))
    } for i in root.findall(".//item")])

# 3. 기부자 정보 조회
def getCntrbtrInfo(page_no=1, num_of_rows=10, data_type='xml', stdr_ym='202402', area_cd='', unity_signgu_cd='', spctr_cd='', cntrbtr_seccd='', cntr_bplc_kndcd='', cntrbtr_nm=''):
    url = f"{BASE_URL}/getCntrbtrInfo"
    params = {
        "serviceKey": SERVICE_KEY,
        "pageNo": page_no,
        "numOfRows": num_of_rows,
        "dataType": data_type,
        "stdrYm": stdr_ym,
        "areaCd": area_cd,
        "unitySignguCd": unity_signgu_cd,
        "spctrCd": spctr_cd,
        "cntrbtrSeccd": cntrbtr_seccd,
        "cntrBplcKndcd": cntr_bplc_kndcd,
        "cntrbtrNm": cntrbtr_nm
    }
    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)
    return pd.DataFrame([{
        "stdrYm": i.findtext("stdrYm"),
        "areaCd": i.findtext("areaCd"),
        "unitySignguCd": i.findtext("unitySignguCd"),
        "spctrCd": i.findtext("spctrCd"),
        "cntrbtrSeccd": i.findtext("cntrbtrSeccd"),
        "cntrBplcKndcd": i.findtext("cntrBplcKndcd"),
        "cntrbtrNm": i.findtext("cntrbtrNm"),
        "cntrAmt": float(i.findtext("cntrAmt", 0)),
        "cntrCo": int(i.findtext("cntrCo", 0)),
        "cntrbtrCo": int(i.findtext("cntrbtrCo", 0))
    } for i in root.findall(".//item")])

# 4. 접수현황 통계데이터 조회
def getRceptStat(page_no=1, num_of_rows=10, data_type='xml', stdr_ym='202402', area_cd='', unity_signgu_cd='', spctr_cd=''):
    url = f"{BASE_URL}/getRceptStat"
    params = {
        "serviceKey": SERVICE_KEY,
        "pageNo": page_no,
        "numOfRows": num_of_rows,
        "dataType": data_type,
        "stdrYm": stdr_ym,
        "areaCd": area_cd,
        "unitySignguCd": unity_signgu_cd,
        "spctrCd": spctr_cd
    }
    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)
    return pd.DataFrame([{
        "stdrYm": i.findtext("stdrYm"),
        "areaCd": i.findtext("areaCd"),
        "unitySignguCd": i.findtext("unitySignguCd"),
        "spctrCd": i.findtext("spctrCd"),
        "rceptAmt": float(i.findtext("rceptAmt", 0)),
        "rceptCo": int(i.findtext("rceptCo", 0))
    } for i in root.findall(".//item")])

# 5. 제공현황 통계데이터 조회
def getProvdStat(page_no=1, num_of_rows=10, data_type='xml', stdr_ym='202402', area_cd='', unity_signgu_cd='', spctr_cd='', food_yn='Y', cnttg_lclas_cd='', cnttg_mlsfc_cd=''):
    url = f"{BASE_URL}/getProvdStat"
    params = {
        "serviceKey": SERVICE_KEY,
        "pageNo": page_no,
        "numOfRows": num_of_rows,
        "dataType": data_type,
        "stdrYm": stdr_ym,
        "areaCd": area_cd,
        "unitySignguCd": unity_signgu_cd,
        "spctrCd": spctr_cd,
        "foodYn": food_yn,
        "cnttgLclasCd": cnttg_lclas_cd,
        "cnttgMlsfcCd": cnttg_mlsfc_cd
    }
    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)
    return pd.DataFrame([{
        "stdrYm": i.findtext("stdrYm"),
        "areaCd": i.findtext("areaCd"),
        "unitySignguCd": i.findtext("unitySignguCd"),
        "spctrCd": i.findtext("spctrCd"),
        "foodYn": i.findtext("foodYn"),
        "cnttgLclasCd": i.findtext("cnttgLclasCd"),
        "cnttgMlsfcCd": i.findtext("cnttgMlsfcCd"),
        "userCo": int(i.findtext("userCo", 0)),
        "provdCo": int(i.findtext("provdCo", 0)),
        "provdAmt": float(i.findtext("provdAmt", 0))
    } for i in root.findall(".//item")])

# 6. 기부물품 현황 조회
def getCnttgInfo(page_no=1, num_of_rows=10, data_type='xml', stdr_ym='202402', distb_tmlmt_use_yn='N', injry_goods_intrcp_yn='Y', food_yn='Y', cnttg_lclas_cd='', cnttg_mlsfc_cd=''):
    url = f"{BASE_URL}/getCnttgInfo"
    params = {
        "serviceKey": SERVICE_KEY,
        "pageNo": page_no,
        "numOfRows": num_of_rows,
        "dataType": data_type,
        "stdrYm": stdr_ym,
        "distbTmlmtUseYn": distb_tmlmt_use_yn,
        "injryGoodsIntrcpYn": injry_goods_intrcp_yn,
        "foodYn": food_yn,
        "cnttgLclasCd": cnttg_lclas_cd,
        "cnttgMlsfcCd": cnttg_mlsfc_cd
    }
    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)
    return pd.DataFrame([{
        "stdrYm": i.findtext("stdrYm"),
        "distbTmlmtUseYn": i.findtext("distbTmlmtUseYn"),
        "injryGoodsIntrcpYn": i.findtext("injryGoodsIntrcpYn"),
        "foodYn": i.findtext("foodYn"),
        "cnttgLclasCd": i.findtext("cnttgLclasCd"),
        "cnttgMlsfcCd": i.findtext("cnttgMlsfcCd"),
        "cnttgQy": int(i.findtext("cnttgQy", 0)),
        "acntbkAmt": float(i.findtext("acntbkAmt", 0))
    } for i in root.findall(".//item")])

# 7. 지원센터 정보 조회
def getSpctrInfo(page_no=1, num_of_rows=10, data_type='xml', stdr_ym='202402', area_cd='', unity_signgu_cd='', spctr_uncd='', spctr_secd='', spctr_cd='', spctr_stscd='', oper_mby_lclas_cd='', oper_mby_sclas_cd='', sttemnt_seccd='', regist_de=''):
    url = f"{BASE_URL}/getSpctrInfo"
    params = {
        "serviceKey": SERVICE_KEY,
        "pageNo": page_no,
        "numOfRows": num_of_rows,
        "dataType": data_type,
        "stdrYm": stdr_ym,
        "areaCd": area_cd,
        "unitySignguCd": unity_signgu_cd,
        "spctrUncd": spctr_uncd,
        "spctrSecd": spctr_secd,
        "spctrCd": spctr_cd,
        "spctrStscd": spctr_stscd,
        "operMbyLclasCd": oper_mby_lclas_cd,
        "operMbySclasCd": oper_mby_sclas_cd,
        "sttemntSeccd": sttemnt_seccd,
        "registDe": regist_de
    }
    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)
    return pd.DataFrame([{
        "stdrYm": i.findtext("stdrYm"),
        "areaCd": i.findtext("areaCd"),
        "unitySignguCd": i.findtext("unitySignguCd"),
        "spctrUncd": i.findtext("spctrUncd"),
        "spctrSecd": i.findtext("spctrSecd"),
        "spctrCd": i.findtext("spctrCd"),
        "spctrTelno": i.findtext("spctrTelno"),
        "spctrAdres": i.findtext("spctrAdres"),
        "spctrDetailAdres": i.findtext("spctrDetailAdres"),
        "spctrStscd": i.findtext("spctrStscd"),
        "operMbyLclasCd": i.findtext("operMbyLclasCd"),
        "operMbySclasCd": i.findtext("operMbySclasCd"),
        "sttemntSeccd": i.findtext("sttemntSeccd"),
        "registDe": i.findtext("registDe"),
        "rceptAmt": float(i.findtext("rceptAmt", 0)),
        "provdAmt": float(i.findtext("provdAmt", 0)),
        "undtakeAmt": float(i.findtext("undtakeAmt", 0)),
        "trnsferAmt": float(i.findtext("trnsferAmt", 0)),
        "userCo": int(i.findtext("userCo", 0))
    } for i in root.findall(".//item")])

# 8. 보유물품 현황 조회
def getPreferInfo(page_no=1, num_of_rows=10, data_type='xml', area_cd='', unity_signgu_cd='', spctr_cd='', prefer_cnttg_clscd=''):
    url = f"{BASE_URL}/getPreferInfo"
    params = {
        "serviceKey": SERVICE_KEY,
        "pageNo": page_no,
        "numOfRows": num_of_rows,
        "dataType": data_type,
        "areaCd": area_cd, # 지역 코드
        "unitySignguCd": unity_signgu_cd, # 통합시군구 코드
        "spctrCd": spctr_cd, # 지원센터 코드
        "preferCnttgClscd": prefer_cnttg_clscd # 물품분류 코드 
    }
    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)
    return pd.DataFrame([{
        "areaCd": i.findtext("areaCd"),
        "unitySignguCd": i.findtext("unitySignguCd"),
        "spctrCd": i.findtext("spctrCd"),
        "preferCnttgClscd": i.findtext("preferCnttgClscd"),
        "holdQy": int(i.findtext("holdQy", 0))
    } for i in root.findall(".//item")])

In [13]:
if __name__ == "__main__":
    for df in [getUserInfo(), getFcltyGrpInfo(), getCntrbtrInfo(), getRceptStat(), getProvdStat(), getCnttgInfo(), getSpctrInfo(), getPreferInfo()]:
        print(df.head())

   stdrYm areaCd unitySignguCd spctrCd userSeccd userClscd happyTrgterYn  \
0  202402     02      02000004  S00201        20        11             N   
1  202402     02      02000004  S00201        10        11             Y   
2  202402     02      02000004  S00201        10        11                 
3  202402     02      02000004  S00201        10        10                 
4  202402     02      02000004  S00201        10        07                 

     useAmt  useCo  userCo  
0   60759.0     14       1  
1   46100.0     12       1  
2   97129.0     25       2  
3  972315.0    236      19  
4  102058.0     26       2  
   stdrYm areaCd unitySignguCd spctrCd fcltySeNm fcltyGrpClscd       useAmt  \
0  202402     02      02000004  S00201      이용시설            11   1364709.00   
1  202402     02      02000004  S00201      기타시설            51   2622120.20   
2  202402     02      02000004  S00201      기타시설            53   2678717.14   
3  202402     03      03000006  S02501      이용시설     

# **전국푸드뱅크및푸드마켓표준데이터**

In [1]:
encode = "UHD0d7Tof2U4mU3gGH%2B3QxTtjc5FwFNnHyqyTBWq%2BnI5TeubdJF6ytylohZXVLp%2B7oVsmoG6CUVmNaqKsyRGIA%3D%3D"
decode = "UHD0d7Tof2U4mU3gGH+3QxTtjc5FwFNnHyqyTBWq+nI5TeubdJF6ytylohZXVLp+7oVsmoG6CUVmNaqKsyRGIA=="

In [2]:
import requests

url = 'http://api.data.go.kr/openapi/tn_pubr_public_food_bank_api'

params ={'serviceKey' : decode, 'pageNo' : '0', 'numOfRows' : '100', 'type' : 'xml', 'CONM_NM' : '대구광역푸드뱅크', 'CNTR_SE' : '', 'CTPV_NM' : '', 'SGG_NM' : '', 'LCTN_ROAD_NM' : '', 'LCTN_LOTNO_ADDR' : '', 'DCLR_SE_NM' : '', 'TELNO' : '', 'OPER_INST_NM' : '', 'DNTN_CMDTY_NM' : '', 'RPRSV_NM' : '', 'TNOEMP' : '', 'DATA_CRTR_YMD' : '', 'instt_code' : '', 'instt_nm' : '' }

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>03</resultCode><resultMsg>NODATA_ERROR</resultMsg></header></response>\r\n'
